# Prepare some things
## Load some modules

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('../')
from models import get_autoencoder_model

In [3]:
# Disable warnings output (TSNE outputs one very time)
import warnings
warnings.filterwarnings('ignore')

## Load the data

In [4]:
data_file = "../DO_NOT_PUBLISH/MetaboData_pseudoID.xlsx"
metabol = pd.read_excel(data_file, sheet_name="metabs", index_col=0).T
# Min-max normalization
metabol.iloc[:,:] = MinMaxScaler().fit_transform(metabol)
metabol.head()

,glycine,alanine,valine,leucine,isoleucine,serine,threonine,proline,arginine,glutamic,...,tcc,m_bu_p,mehp,x4_hpb,ibp,cibp,apap,apap_g,dcf,hctz
S000178_1,0.203059,0.111499,0.191536,0.250043,0.257227,0.298541,0.066532,0.136444,0.188422,0.174716,...,0.0,0.271801,0.266637,0.161973,0.0,0.0,0.018271,0.028760,0.296658,0.0
S000443_1,0.073279,0.115869,0.132288,0.149303,0.148693,0.084232,0.022798,0.092352,0.025114,0.074864,...,0.0,0.221467,0.197153,0.129863,0.0,0.0,0.009566,0.010220,0.199715,0.0
S000491_1,0.212805,0.304089,0.249373,0.263820,0.277999,0.385945,0.242866,0.262947,0.085272,0.116247,...,0.0,0.431987,0.480285,0.612164,0.0,0.0,0.030228,0.060776,0.349946,0.0
S000732_1,0.136319,0.715659,0.582759,0.654555,0.671802,0.386421,0.549783,0.517717,0.404796,0.199605,...,0.0,0.168995,0.150266,0.189479,0.0,0.0,0.001626,0.004522,0.348757,0.0
S000732_2,0.178756,0.387277,0.334169,0.344584,0.369569,0.187183,0.546836,0.228796,0.201617,0.165235,...,0.0,0.123065,0.164727,0.188603,0.0,0.0,0.081787,0.212806,0.369097,0.0


In [5]:
samples = pd.read_excel(data_file, sheet_name="samples", index_col=0)

# Drop the observations cointaining missing values:
drop_index = samples[samples['CVrisk3'].isna()].index
metabol.drop(drop_index, inplace=True, errors="ignore")
# Subset only the indeces present in the metabol data set
samples = samples.loc[metabol.index,:]

# Code gender as 0, 1
samples['gender'] = samples['gender'] -1
# Code CVrisk3 as 0, 1, 2
samples['CVrisk3'] = pd.Categorical(samples['CVrisk3'], categories=('Low','Med','Hig')).codes
# Convert time_origin to categorical type
samples['time_origin'] = pd.Categorical(samples['time_origin'], categories=('baseline','six','twelve'))

samples.head()

,X.SampleID,KKHNGID,time_origin,gender,pred_risk,CVrisk3,AGE,SCBIA1,SCANT2,SCBIA4,TG,HDL,LDL,CHO,HBA1CPC,SCBP1,SCBP2,HSCRP,CR
S000178_1,S000178_1,20161,baseline,0,0.020379,2,51,22.52,90.0,10.15,1.13,1.68,4.94,6.87,5.1,125,90,0.71,77.0
S000443_1,S000443_1,3557,baseline,1,0.000580,0,41,23.06,96.8,33.33,1.30,1.68,2.78,4.29,5.5,113,78,0.32,63.0
S000491_1,S000491_1,1783,baseline,1,0.001671,1,47,22.15,92.3,30.20,0.90,1.46,2.80,4.69,5.3,117,82,0.78,65.0
S000732_1,S000732_1,104735,baseline,1,0.000308,0,38,22.90,87.6,30.42,1.38,1.94,1.56,3.54,5.2,109,83,2.86,70.0
S000732_2,S000732_2,104735,six,1,0.000398,0,39,23.34,102.5,31.41,1.10,1.86,1.65,3.82,4.9,117,83,2.13,78.0


In [6]:
print("Number of observations:", metabol.shape[0])
print("Number of variables:", metabol.shape[1])
print("Number of data points:", np.multiply(*metabol.shape))

Number of observations: 1020
Number of variables: 411
Number of data points: 419220


## Load the health variabes

# Try AE with different hyperparameters

In [7]:
LEARNING_RATE = 1
MOMENTUM = 0.8
optimizer = keras.optimizers.SGD(learning_rate=LEARNING_RATE, momentum=MOMENTUM)

INPUT_DIM = metabol.shape[1]
LATENT_DIM = 5
N_CLUSTERS = LATENT_DIM

num_data_points = np.multiply(*metabol.shape)

## Intermediate layers number and dimentions

Let's try different numbers and dimentions of intermediate layers:

In [16]:
EPOCHS = 50
BATCH_SIZE = 32
INTERMEDIATE_DIMS = [(512, 512, 2048), # same as in MNIST, for reference
                     (64, 32),
                     (32, 16),
                     (16, 4),
                     (16, 16, 64),
                     (16, 16, 128),
                     (16, 16, 256),
                     (32, 32, 64),
                     (32, 32, 128),
                     (32, 32, 256)
                    ]

# Try every combination of dimention 5 times, 
# then get the mean of the results
for dims in INTERMEDIATE_DIMS:
    loss = []
    val_loss = []
    loss_ratio = []
    for i in np.arange(5):
        model_ae = get_autoencoder_model(INPUT_DIM, LATENT_DIM, dims)
        data_params_ratio = num_data_points / (model_ae.encoder.count_params() + 
                                               model_ae.decoder.count_params())
        if i == 0:
            print(f'{dims} [data/params ratio: {data_params_ratio:.1f}]', end=" ")

        model_ae.compile(optimizer=optimizer, loss="mse")
        history = model_ae.fit(metabol, metabol,
                               epochs=EPOCHS,
                               batch_size=BATCH_SIZE,
                               validation_split=0.2,
                               verbose=0)
        loss.append(history.history['loss'][-1])
        val_loss.append(history.history['val_loss'][-1])
        loss_ratio.append(loss[-1] / val_loss[-1])

    print(f"[loss: {np.mean(loss):.4f}, val_loss: {np.mean(val_loss):.4f}, ratio: {np.mean(loss_ratio):.4f}]")

(512, 512, 2048) [data/params ratio: 0.1] [loss: 0.0099, val_loss: 0.0100, ratio: 0.9941]
(64, 32) [data/params ratio: 7.3] [loss: 0.0101, val_loss: 0.0101, ratio: 0.9934]
(32, 16) [data/params ratio: 15.0] [loss: 0.0103, val_loss: 0.0103, ratio: 0.9940]
(16, 4) [data/params ratio: 30.4] [loss: 0.0121, val_loss: 0.0119, ratio: 1.0191]
(16, 16, 64) [data/params ratio: 24.7] [loss: 0.0122, val_loss: 0.0119, ratio: 1.0213]
(16, 16, 128) [data/params ratio: 21.2] [loss: 0.0116, val_loss: 0.0114, ratio: 1.0104]
(16, 16, 256) [data/params ratio: 16.5] [loss: 0.0123, val_loss: 0.0120, ratio: 1.0205]
(32, 32, 64) [data/params ratio: 12.4] [loss: 0.0112, val_loss: 0.0111, ratio: 1.0072]
(32, 32, 128) [data/params ratio: 10.9] [loss: 0.0110, val_loss: 0.0109, ratio: 1.0072]
(32, 32, 256) [data/params ratio: 8.7] [loss: 0.0113, val_loss: 0.0112, ratio: 1.0081]


The best loss is achieved with only two intermediate layers (64,32), but the data/params ratio is very low and this can make the model easily overfit.

There are three configurations that get better a data/params ratio and achieve a similar loss:
- (32, 16)
- (16, 16, 64)
- (16, 16, 128)

The difference in loss can be minimized by training for more epochs:

In [17]:
EPOCHS = 200
INTERMEDIATE_DIMS = [(32, 16),
                     (16, 16, 64),
                     (16, 16, 128),
                    ]

# Try every combination of dimention 5 times, 
# then get the mean of the results
for dims in INTERMEDIATE_DIMS:
    loss = []
    val_loss = []
    loss_ratio = []
    for i in np.arange(5):
        model_ae = get_autoencoder_model(INPUT_DIM, LATENT_DIM, dims)
        data_params_ratio = num_data_points / (model_ae.encoder.count_params() + 
                                               model_ae.decoder.count_params())
        if i == 0:
            print(f'{dims} [data/params ratio: {data_params_ratio:.1f}]', end=" ")

        model_ae.compile(optimizer=optimizer, loss="mse")
        history = model_ae.fit(metabol, metabol,
                               epochs=EPOCHS,
                               batch_size=BATCH_SIZE,
                               validation_split=0.2,
                               verbose=0)
        loss.append(history.history['loss'][-1])
        val_loss.append(history.history['val_loss'][-1])
        loss_ratio.append(loss[-1] / val_loss[-1])

    print(f"[loss: {np.mean(loss):.4f}, val_loss: {np.mean(val_loss):.4f}, ratio: {np.mean(loss_ratio):.4f}]")

(32, 16) [data/params ratio: 15.0] [loss: 0.0094, val_loss: 0.0095, ratio: 0.9916]
(16, 16, 64) [data/params ratio: 24.7] [loss: 0.0096, val_loss: 0.0096, ratio: 0.9968]
(16, 16, 128) [data/params ratio: 21.2] [loss: 0.0095, val_loss: 0.0096, ratio: 0.9939]


I'll select the combination (16, 16, 64), since it achieves the better relation between data/params ratio and loss.

## Batch size

Let's now try different batch sizes to see if there is are any big differences:

In [18]:
EPOCHS = 50
INTERMEDIATE_DIMS = (16, 16, 64)
BATCH_SIZES = (16, 32, 64, 128)

# Try every batch size 5 times, 
# then get the mean of the results
for batch_size in BATCH_SIZES:
    loss = []
    val_loss = []
    loss_ratio = []
    print(f'batch size: {batch_size}', end=" ")
    for i in np.arange(5):
        model_ae = get_autoencoder_model(INPUT_DIM, LATENT_DIM, INTERMEDIATE_DIMS)
        model_ae.compile(optimizer=optimizer, loss="mse")
        history = model_ae.fit(metabol, metabol,
                               epochs=EPOCHS,
                               batch_size=batch_size,
                               validation_split=0.2,
                               verbose=0)
        loss.append(history.history['loss'][-1])
        val_loss.append(history.history['val_loss'][-1])
        loss_ratio.append(loss[-1] / val_loss[-1])

    print(f"[loss: {np.mean(loss):.4f}, val_loss: {np.mean(val_loss):.4f}, ratio: {np.mean(loss_ratio):.4f}]")

batch size: 16 [loss: 0.0102, val_loss: 0.0102, ratio: 0.9980]
batch size: 32 [loss: 0.0120, val_loss: 0.0118, ratio: 1.0187]
batch size: 64 [loss: 0.0125, val_loss: 0.0122, ratio: 1.0258]
batch size: 128 [loss: 0.0127, val_loss: 0.0124, ratio: 1.0264]


The smaller batch sizes achieve a lower loss, but also the training process takes longer.

Both things can be contrarrested by selecting a different amount of epochs. So it does'nt seem to be criticall.

I'll select a batch size of 32 and vary the number of epochs depending on how long it takes for the loss to converge.

# Selected parameters

Based on the results, I decided to select the following parameters:
- Intermediate dimentions: (16, 16, 64) *
- Batch size: 32

*\* Finally, I found that the deep clustering models performed better with **two intermadiate layers**, with dimentions **(16, 32)** .*

In [13]:
model_ae = get_autoencoder_model(INPUT_DIM, LATENT_DIM, (16, 32))
data_params_ratio = num_data_points / (model_ae.encoder.count_params() + 
                                       model_ae.decoder.count_params())
print(f"Data / params ratio: {data_params_ratio:.2f}")

Data / params ratio: 27.93
